In [22]:
from utils.openai_client import get_chat_completion
from utils.function_calling import get_response, get_tool_function
from tools.tools_schema import TOOLS_SCHEMA
from openai import OpenAI
import os

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

system_prompt = """You are a helpful assistant for Duke University. You answer questions about Duke's programs, courses, professors, events, and related information using a set of tools. Be thorough, polite, and accurate. If something is unclear, ask follow-up questions to get more context.

Guidelines:
1. **If a user query is vague or underspecified**, do not assume. Instead, ask follow-up questions. For example, if someone asks "Who is Dr. Smith?" ask what program or department they're in before calling a professor-related tool.
2. **Use tools only when needed**. Think through the query: Is it about a program, course, professor, or event? Then pick the tool that best fits that need. If a name match seems fuzzy or incorrect, avoid responding with false confidence.
3. **Avoid inaccurate tool calls**: For example, if a professor's name does not have an exact match, don't return information about someone with a similar name. Instead, say "I couldn't find a match — could you clarify which program they're associated with?"
4. **For professor-related questions**, try to get their department or program (e.g., AIPI or MEM since you have tools for both) before selecting a tool. Some professors are only listed in specific databases. 
5. **Use the web search tool** as a fallback for Duke-related queries that don't fit neatly into any other tool (e.g., "What is Dr. X researching currently?" or "What does Duke's housing policy look like?") OR if the data you get back from the other tools is not helpful, instead of saying "I dont know" try to use the web search tool to answer the question. Incase web search doesn't work either, you can use your own knowledge to answer the question.
6. You are **not** allowed to answer questions that are not related to Duke University. This is very important. If a user query is **not related to Duke University**, respond by saying it's out of scope and that you're here to help with Duke-related questions.
7. Always follow safe and ethical practices when answering questions.
8. Provide links, references, and citations when relevant.
9. !!!!!!! When asked about the Artificial Intelligence for Product Innovation or AIPI course make sure to use the get_courses tool in your planning !!!!!!!!!
10. Use "pratt_search" tool to get general information about Pratt School of Engineering and their programs please!.

Be concise when appropriate, but offer long, elaborate answers when more detail would be helpful.
"""

user_query = "Who is brinnae bent from the AIPI program?"

tools = TOOLS_SCHEMA
tool_choice = 'auto'

messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_query}]

# Build request kwargs conditionally
kwargs = {
        "model": "gpt-4o",
        "messages": messages,
        "temperature": 0.1
    }

if tools:
        kwargs["tools"] = tools
        kwargs["tool_choice"] = tool_choice

try:
        response = client.chat.completions.create(**kwargs)
        message = response.choices[0].message
        print("✅ Assistant:\n", message.content)

        if message.tool_calls:
            print("\n🛠️ Tool Calls:")
            for call in message.tool_calls:
                print(f"  Tool: {call.function.name}")
                print(f"  Args: {call.function.arguments}")
        print(message)
except Exception as e:
        print(f"❌ OpenAI API call failed: {e}")


✅ Assistant:
 None

🛠️ Tool Calls:
  Tool: get_AIPI_details
  Args: {"query":"Brinnae Bent"}
ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_fKzwgeiOvD0mOHutFqFXzrzU', function=Function(arguments='{"query":"Brinnae Bent"}', name='get_AIPI_details'), type='function')])


In [25]:
import os
import json
from openai import OpenAI
from utils.openai_client import get_chat_completion
from tools.tools_schema import TOOLS_SCHEMA
from utils.function_calling import get_tool_function, tool_status_messages

# Initialize client
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

def simulate_tool_loop(user_query):
    system_prompt = """You are a helpful assistant for Duke University. You answer questions about Duke's programs, courses, professors, events, and related information using a set of tools. Be thorough, polite, and accurate. If something is unclear, ask follow-up questions to get more context.

Guidelines:
1. **If a user query is vague or underspecified**, do not assume. Instead, ask follow-up questions. For example, if someone asks "Who is Dr. Smith?" ask what program or department they're in before calling a professor-related tool.
2. **Use tools only when needed**. Think through the query: Is it about a program, course, professor, or event? Then pick the tool that best fits that need. If a name match seems fuzzy or incorrect, avoid responding with false confidence.
3. **Avoid inaccurate tool calls**: For example, if a professor's name does not have an exact match, don't return information about someone with a similar name. Instead, say "I couldn't find a match — could you clarify which program they're associated with?"
4. **For professor-related questions**, try to get their department or program (e.g., AIPI or MEM since you have tools for both) before selecting a tool. Some professors are only listed in specific databases. 
5. **Use the web search tool** as a fallback for Duke-related queries that don't fit neatly into any other tool (e.g., "What is Dr. X researching currently?" or "What does Duke's housing policy look like?") OR if the data you get back from the other tools is not helpful, instead of saying "I dont know" try to use the web search tool to answer the question. Incase web search doesn't work either, you can use your own knowledge to answer the question.
6. You are **not** allowed to answer questions that are not related to Duke University. This is very important. If a user query is **not related to Duke University**, respond by saying it's out of scope and that you're here to help with Duke-related questions.
7. Always follow safe and ethical practices when answering questions.
8. Provide links, references, and citations when relevant.
9. !!!!!!! When asked about the Artificial Intelligence for Product Innovation or AIPI course make sure to use the get_courses tool in your planning !!!!!!!!!
10. Use "pratt_search" tool to get general information about Pratt School of Engineering and their programs please!.

Be concise when appropriate, but offer long, elaborate answers when more detail would be helpful.
"""  # truncated for brevity

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_query}
    ]

    max_cycles = 5
    for i in range(max_cycles):
        print(f"\n🔁 Iteration {i+1} --------------------")
        
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=messages,
            temperature=0.1,
            tools=TOOLS_SCHEMA,
            tool_choice="auto"
        )

        message = response.choices[0].message

        # ✅ If the model returns a content response (no tool call)
        if message.content:
            print(f"🧠 Assistant:\n{message.content}")
            messages.append({"role": "assistant", "content": message.content})
            break  # Done

        # ✅ If there's a tool call
        if message.tool_calls:
            tool_call = message.tool_calls[0]
            tool_name = tool_call.function.name
            tool_args = json.loads(tool_call.function.arguments)

            print(f"🛠️ Tool Call: {tool_name}({tool_args})")
            print(tool_status_messages.get(tool_name, "Running tool..."))

            # Run tool
            tool_func = get_tool_function(tool_name)
            tool_response = tool_func(**tool_args)

            # Append assistant's tool call message
            messages.append({
                "role": "assistant",
                "content": None,
                "tool_calls": [{
                    "id": tool_call.id,
                    "type": "function",
                    "function": {
                        "name": tool_name,
                        "arguments": tool_call.function.arguments
                    }
                }]
            })

            # Append tool's response message
            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": str(tool_response)
            })

        else:
            print("⚠️ No tool call or assistant content. Exiting.")
            break


In [27]:
simulate_tool_loop("Can you get events on zoom for the next 2 days?")


🔁 Iteration 1 --------------------


2025-06-13 12:22:23.422 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


🛠️ Tool Call: get_events({'query': 'zoom'})
Getting events...


AttributeError: st.session_state has no attribute "client". Did you forget to initialize it? More info: https://docs.streamlit.io/develop/concepts/architecture/session-state#initialization